In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
# 设置数据转换
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载MNIST数据集
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(1, 11):  # 总共训练10个epochs
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.320864
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.039037
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.209418
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.717672
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.643995
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.686166
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.451286
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.443755
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.596283
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.417277

Test set: Average loss: 0.1920, Accuracy: 9422/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.698409
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.367976
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.847180
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.348354
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.299443
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.318427
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.242325
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.349813
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.213626
T

In [13]:
model.eval()
with torch.no_grad():
    temp_dataset = datasets.MNIST('./data', train=False)
    print(type(temp_dataset[0][0]), temp_dataset[0][0])
    print(transforms.ToTensor()(temp_dataset[0][0]).shape)

<class 'PIL.Image.Image'> <PIL.Image.Image image mode=L size=28x28 at 0x2DAC94EB1D0>
torch.Size([1, 28, 28])


In [21]:
type(temp_dataset[0][1])
for i, data in enumerate(train_loader, 0):
    inputs, labels = data
    print(i, labels.shape)
    print(inputs.shape)
    break

0 torch.Size([64])
torch.Size([64, 1, 28, 28])
